---
---

<center><h1> Project: Extract Transform Load.</h1></center>

---

----




![](images/pipeline.png)


---

In [1]:
# importing the required libraries
import schedule
import time
import pandas as pd
import mysql.connector as mysql
from datetime import timedelta
import datetime

In [2]:
def create_connection():
    db = mysql.connect(
    host = "localhost",
    user = "lakshay", ## Enter your username here
    password = "ABC@123", ## Enter your password here
    database = "website", ## Enter your database name here
    auth_plugin = "mysql_native_password",
) 
   
    return db

---
---

#### `EXTRACT`


---
---

In [3]:
# define the extract function
def extract(db):
    
    # create database cursor
    cursor = db.cursor()
    
    # current time & time 5 minutes before
    now = datetime.datetime.now()
    now_minus_5 = now - datetime.timedelta(minutes = 10)
    
    print("Extracting results between {} and {}".format(str(now_minus_5),str(now)))
    
    # command to extract the data of last 5 minutes.
    command = "SELECT * FROM users WHERE signup_time BETWEEN '{}' AND '{}'".format(now_minus_5, now)
    
    # execute the command and return the results.
    cursor.execute(command)
    data = cursor.fetchall()
    return now_minus_5, now, pd.DataFrame.from_records(data, columns= ['user_id',
                                                     'user_email',
                                                     'user_name',
                                                     'source',
                                                     'is_prime',
                                                     'signup_time'])

---

#### `Transform`

---

In [4]:
# define the tranform function
def transform(df, start_time, end_time):
    print("Transformation")
    df.source.replace("Not Available", "Organic", inplace=True)
    
    source_count = df.groupby(['source'])['user_id'].count()
    prime_count = df.groupby(['source'])['is_prime'].sum()
    
    source_count_dict = source_count.to_dict()
    for key in prime_count.to_dict():
        new_key_name = "prime_from_" + key
        source_count_dict[new_key_name] = prime_count[key]
    
    source_count_dict['start_time'] = str(start_time)
    source_count_dict['end_time'] = str(end_time)
    
    
    return source_count_dict

---

#### `LOAD`

---

In [5]:
def load(source_count_dict, db):
    print("Loading")
    cursor = db.cursor()
    command = "INSERT INTO signup_summary({col}) values{val}".format(col= ','.join(source_count_dict.keys()),
                                                                       val= tuple(source_count_dict.values()))
    cursor.execute(command)
    db.commit()

---

#### Define the ETL Pipeline

- Step 1: Extraction
- Step 2: Transformation
- Step 3: Loading   

---

In [6]:
def job():

    db_object = create_connection()
    
    ## STEP 1: Extract
    start_time, end_time, df = extract(db_object)
    
    ## Step 2: Transform
    final_results = transform(df, start_time, end_time)
    
    ## Step 3: Load
    load(final_results, db_object)

In [7]:
schedule.every(300).seconds.do(job)

Every 300 seconds do job() (last run: [never], next run: 2021-09-09 19:38:09)

In [8]:
while True:
        schedule.run_pending() 
        time.sleep(1)

Extracting results between 2021-09-10 10:47:03.509310 and 2021-09-10 10:57:03.509310
Transformation
Loading


ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ',Email,Facebook,Instagram,LinkedIn,Organic,prime_from_,prime_from_Email,prime_fr' at line 1

In [342]:
while True:
        job()
        time.sleep(300)

Extracting results between 2021-09-09 19:11:20.077823 and 2021-09-09 19:16:20.077823


KeyboardInterrupt: 

In [1]:
import logging

In [3]:
logging.basicConfig(filename="mylogfile.log",  level=logging.INFO)

In [6]:
for i in range(10):
    logging.info(msg= "You have a new log entry!!")